<a href="https://colab.research.google.com/github/swilsonmfc/pandas/blob/main/7_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Bookings

![](https://cdn.inprnt.com/thumbs/5b/fa/5bfa6a8ea6c461d1c21c0a44962968ed.jpg?response-cache-control=max-age=2628000)

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.graphics.regressionplots as regplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data
* https://www.kaggle.com/jessemostipak/hotel-booking-demand

In [ ]:
!wget -O hotel.zip https://github.com/swilsonmfc/pandas/blob/main/hotel.zip?raw=true

--2021-07-08 21:30:33--  https://github.com/swilsonmfc/pandas/blob/main/hotel.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/swilsonmfc/pandas/raw/main/hotel.zip [following]
--2021-07-08 21:30:33--  https://github.com/swilsonmfc/pandas/raw/main/hotel.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/swilsonmfc/pandas/main/hotel.zip [following]
--2021-07-08 21:30:33--  https://raw.githubusercontent.com/swilsonmfc/pandas/main/hotel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1308365 (1.2M) [applic

In [ ]:
!unzip hotel.zip

Archive:  hotel.zip
replace hotel_bookings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hotel_bookings.csv      


In [ ]:
hotel_df = pd.read_csv('./hotel_bookings.csv')

In [ ]:
hotel_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


## Types

In [ ]:
hotel_df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

## Missing
* Missing data here seems reasonable for the most part
* 4 missing children - Impute to median

In [ ]:
hotel_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [ ]:
# Series where method vs np.where
hotel_df['children'] = hotel_df.children.where(hotel_df.children.notna(), 0)
hotel_df['country']  = hotel_df.country.fillna('UNKNOWN')
hotel_df['agent']    = hotel_df.agent.fillna('NONE')
hotel_df['company']  = hotel_df.company.fillna('NONE')

In [ ]:
hotel_df.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

## Outliers
* More than 10 people booked in a room
  * All cancelled
  * Agent 96 is present on 1/2 of them
* The hotel paid one person to stay there
  * Best to truncate that to $0
  * One room cost \$5,400

In [ ]:
hotel_df = hotel_df[(hotel_df.adr >= 0) & (hotel_df.adr < 1000)]

# Linear Regression

In [ ]:
f = 'adr ~ hotel + arrival_date_month'
model = smf.ols(formula=f, data=hotel_df)
fitted = model.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    adr   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     2812.
Date:                Thu, 08 Jul 2021   Prob (F-statistic):               0.00
Time:                        21:38:42   Log-Likelihood:            -6.1710e+05
No. Observations:              119388   AIC:                         1.234e+06
Df Residuals:                  119375   BIC:                         1.234e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [ ]:
f = """adr ~ + C(arrival_date_month, Treatment(reference='January'))
             + C(hotel, Treatment(reference='Resort Hotel'))""" 
model = smf.ols(formula=f, data=hotel_df)
fitted = model.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    adr   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     2812.
Date:                Thu, 08 Jul 2021   Prob (F-statistic):               0.00
Time:                        21:38:43   Log-Likelihood:            -6.1710e+05
No. Observations:              119388   AIC:                         1.234e+06
Df Residuals:                  119375   BIC:                         1.234e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [ ]:
pred = pd.DataFrame([{'hotel': 'City Hotel', 'arrival_date_month': 'July'}])
pred

,hotel,arrival_date_month
0,City Hotel,July


In [ ]:
predictions = fitted.get_prediction(pred)
pred_df = predictions.summary_frame()
pred_df = pred_df.reset_index(drop=True)
pred_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,130.191011,0.38948,129.427637,130.954386,46.851099,213.530923


# Assignment
* Try various combinations of features 
  * Note the R-Squared Value
  * Attempt to get a score over .5
* Does your model seem like a good fit?
  * Do the coefficients make sense?
  * Make a prediction of your model
* How can we continue to improve on the model?
  * Can you engineer additional features that may work well?
  * What should we do with features that have poor P-Values?
